## Youtube Channel ID Converter

This Python code implements a YouTube channel ID converter. It takes URLs, such as those provided by a third-party tool, and processes them to extract and convert the URL into the corresponding YouTube channel name.

In [ ]:
# Test this code to see if I can turn one Channel ID and get the the proper Name

from googleapiclient.discovery import build

# Set up your API key and build the service
api_key = 'ADD_YOUR_KEY'
youtube = build('youtube', 'v3', developerKey=api_key)

# Get channel details using channel ID
channel_id = 'ADD_CHANNEL_ID'
response = youtube.channels().list(
    part='snippet',
    id=channel_id
).execute()

# Extract handle
if 'items' in response and response['items']:
    handle = response['items'][0]['snippet'].get('customUrl')  # Old way
    title = response['items'][0]['snippet']['title']
    print(f"Channel Title: {title}")
    if handle:
        print(f"Custom URL: https://www.youtube.com/@{handle}")
    else:
        print("This channel does not have a custom handle.")
else:
    print("Channel not found.")

The code below will process a batch of URLs, convert them, and save the results in a CSV file.

In [ ]:
# Run this code for a batch of Channel ID

import pandas as pd
from googleapiclient.discovery import build

# Replace this with your actual YouTube Data API v3 key
api_key = 'ADD_YOUR_KEY'

# Build YouTube API service
youtube = build('youtube', 'v3', developerKey=api_key)

# Load CSV file (no header, just URLs)
df = pd.read_csv(r'âth\channel_id_list.csv', on_bad_lines='skip', header=None)
df.columns = ['channel_url']  # Rename for clarity

# Create a clean channel_id column by stripping the URL prefix
df['channel_id'] = df['channel_url'].astype(str).str.replace(
    'https://www.youtube.com/channel/', '', regex=False
).str.strip()

# Prepare results list
original_urls = []
channel_ids = []
channel_titles = []
custom_urls = []

# Loop through each channel ID
for i, row in df.iterrows():
    original_url = row['channel_url']
    channel_id = row['channel_id']

    try:
        response = youtube.channels().list(
            part='snippet',
            id=channel_id
        ).execute()

        if 'items' in response and response['items']:
            snippet = response['items'][0]['snippet']
            title = snippet.get('title', 'N/A')
            handle = snippet.get('customUrl')
            custom_url = f"https://www.youtube.com/{handle}" if handle else "No handle"
        else:
            title = 'Not found'
            custom_url = 'N/A'

    except Exception as e:
        title = 'Error'
        custom_url = str(e)

    # Append to lists
    original_urls.append(original_url)
    channel_ids.append(channel_id)
    channel_titles.append(title)
    custom_urls.append(custom_url)

    print(f"{channel_id}: {title} | {custom_url}")

# Create result DataFrame
results_df = pd.DataFrame({
    'original_url': original_urls,
    'channel_id': channel_ids,
    'channel_title': channel_titles,
    'custom_url': custom_urls
})

# Save to CSV (optional)
results_df.to_csv('channel_info_output.csv', index=False)

results_df
